In [1]:
import os
import json
import re
import scipy.io as spio
import numpy as np
from json_tricks.np import dump, dumps, load, loads
import pandas as pd
import pymworks
import cPickle as pkl
from collections import Counter

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

In [3]:
source = '/nas/volume1/2photon/projects'
experiment = 'gratings_phaseMod' #'retino_bar' #'gratings_phaseMod'
session = '20170927_CE059' #'20170902_CE054' #'20170825_CE055'
acquisition = 'FOV1_zoom3x' #'FOV1_zoom3x_run2' #'FOV1_planar'
functional_dir = 'functional_subset'
acquisition_dir = os.path.join(source, experiment, session, acquisition)


In [4]:
# Load reference info:
ref_json = 'reference_%s.json' % functional_dir 
with open(os.path.join(acquisition_dir, ref_json), 'r') as fr:
    ref = json.load(fr)

# Load SI meta data:
si_basepath = ref['raw_simeta_path'][0:-4]
simeta_json_path = '%s.json' % si_basepath
with open(simeta_json_path, 'r') as fs:
    simeta = json.load(fs)

file_names = sorted([k for k in simeta.keys() if 'File' in k], key=natural_keys)
nfiles = len(file_names)


In [5]:
# Get PARADIGM INFO:
path_to_functional = os.path.join(acquisition_dir, functional_dir)
paradigm_dir = 'paradigm_files'
path_to_paradigm_files = os.path.join(path_to_functional, paradigm_dir)
path_to_raw = os.path.join(path_to_paradigm_files, 'raw')

# Get SERIAL data:
serialdata_fns = os.listdir(path_to_raw)
serialdata_fns = sorted([f for f in serialdata_fns if 'serial_data' in f], key=natural_keys)
print "Found %02d serial-data files, and %03d TIFFs." % (len(serialdata_fns), nfiles)

# Load MW info:
pydict_jsons = os.listdir(path_to_paradigm_files)
pydict_jsons = sorted([p for p in pydict_jsons if p.endswith('.json') and 'trial_info' in p], key=natural_keys)
print "Found %02d MW files, and %02d ARD files." % (len(pydict_jsons), len(serialdata_fns))

Found 03 serial-data files, and 003 TIFFs.
Found 03 MW files, and 03 ARD files.


In [22]:
fid = 0
fn = serialdata_fns[fid]
print fn

20170927_CE059_FOV1_zoom3x_003_serial_data_20170927185244245420.txt


In [6]:
trialdict_by_file = dict()
#for fid,fn in enumerate(sorted(serialdata_fns, key=natural_keys)):
# fid = 0
# fn = serialdata_fns[fid]
framerate = float(simeta['File001']['SI']['hRoiManager']['scanFrameRate'])

currfile = "File%03d" % int(fid+1)

print "================================="
print "Processing files:"
print "MW: ", pydict_jsons[fid]
print "ARD: ", serialdata_fns[fid]
print "---------------------------------"

### LOAD MW DATA.
with open(os.path.join(path_to_paradigm_files, pydict_jsons[fid]), 'r') as f:
    trials = json.load(f)

### LOAD SERIAL DATA.
ardata = pd.read_csv(os.path.join(path_to_raw, serialdata_fns[fid]), sep='\t')
print ardata.columns

frame_triggers = ardata[' frame_trigger']
arduino_time = ardata[' relative_arduino_time']
bitcodes = ardata[' pixel_clock']

frame_on_idxs = [idx+1 for idx,diff in enumerate(np.diff(frame_triggers)) if diff==1]
#print len(frame_on_idxs)
frame_on_idxs.append(0)
frame_on_idxs = sorted(frame_on_idxs)
print "Found %i frame-triggers:" % len(frame_on_idxs)

Processing files:
MW:  trial_info_20170927_CE059_FOV1_zoom3x_003.json
ARD:  20170927_CE059_FOV1_zoom3x_003_serial_data_20170927185244245420.txt
---------------------------------
Index([u'acquisition_trigger', u' frame_trigger', u' pixel_clock',
       u' abosolute_arduino_time', u' relative_arduino_time',
       u'     absolute_computer_time', u' relative_computer_time'],
      dtype='object')
Found 12625 frame-triggers:


In [23]:
[i for i,b in enumerate(bitcodes) if b==12][0]

4099

In [77]:
### Get bitcodes for each frame:
frame_bitcodes = dict()
for idx,frameidx in enumerate(frame_on_idxs):
    #framenum = 'frame'+str(idx)
    if idx==len(frame_on_idxs)-1:
        bcodes = bitcodes[frameidx:]
    else:
        bcodes = bitcodes[frameidx:frame_on_idxs[idx+1]]
    frame_bitcodes[idx] = bcodes

### Get first frame of trial start:
modes_by_frame = dict()
for frame in frame_bitcodes.keys():
    bitcode_counts = Counter(frame_bitcodes[frame])
    modes_by_frame[frame] = bitcode_counts.most_common(1)[0][0]

first_stim_frame = [k for k in sorted(modes_by_frame.keys()) if modes_by_frame[k]>0][0]

### Get all bitcodes and corresonding frame-numbers for each trial:
trialdict = dict()
allframes = sorted(frame_bitcodes.keys()) #, key=natural_keys)
curr_frames = sorted(allframes[first_stim_frame+1:]) #, key=natural_keys)


In [78]:
print first_stim_frame
frame_bitcodes[349]

7


4091    15
4092    15
4093    15
4094    15
4095    15
4096    15
4097    15
4098    13
4099    12
4100    12
4101    12
4102    12
Name:  pixel_clock, dtype: int64

In [92]:

for trial in ['5']: #sorted(trials.keys(), key=natural_keys): #sorted(trials.keys, key=natural_keys):
    #print trial
    trialdict[trial] = dict()
    trialdict[trial]['name'] = trials[trial]['stimuli']['stimulus']
    trialdict[trial]['duration'] = trials[trial]['stim_off_times'] - trials[trial]['stim_on_times']

    first_found_frame = []
    minframes = 4
    for bitcode in trials[trial]['all_bitcodes']:
            looking = True
            while looking is True:
                for frame in sorted(curr_frames):
                    tmp_frames = [i for i in frame_bitcodes[frame] if i==bitcode]
                    consecutives = [i for i in np.diff(tmp_frames) if i==0]

                    if frame>1:
                        tmp_frames_pre = [i for i in frame_bitcodes[int(frame)-1] if i==bitcode]
                        consecutives_pre = [i for i in np.diff(tmp_frames_pre) if i==0]

                    if frame>1 and len(consecutives_pre)>=minframes:
                        if len(consecutives_pre) > len(consecutives):
                            first_frame = int(frame) - 1
                        elif len(consecutives)>=minframes:
                            first_frame = int(frame)
                        #print "found2...", bitcode, first_frame #len(curr_frames)
                        looking = False

                    elif len(consecutives)>=minframes:
                        first_frame = frame 
                        #print "found...", bitcode, first_frame #len(curr_frames)
                        looking = False

                    if looking is False:
                        break

            first_found_frame.append((bitcode, first_frame)) #first_frame))
            curr_frames = allframes[first_frame+1:] #curr_frames[idx:] #curr_frames[first_frame:]

    #if (first_found_frame[-1][1] - first_found_frame[0][1])/framerate > 2.5:
    #print "Trial %i dur (s):" % int(trial)
    print (first_found_frame[-1][1] - first_found_frame[0][1])/framerate, '[Tial %i]' % int(trial)

    trialdict[trial]['stim_on_idx'] = first_found_frame[0][1]
    trialdict[trial]['stim_off_idx'] = first_found_frame[-1][1]

1.99325108632 [Tial 5]


In [93]:
first_found_frame

[(2, 2403),
 (4, 2404),
 (11, 2405),
 (7, 2407),
 (6, 2408),
 (5, 2410),
 (7, 2411),
 (5, 2413),
 (8, 2414),
 (13, 2415),
 (4, 2417),
 (2, 2418),
 (0, 2420),
 (5, 2421),
 (2, 2423),
 (1, 2424),
 (11, 2425),
 (3, 2427),
 (14, 2428),
 (5, 2430),
 (11, 2431),
 (12, 2433),
 (6, 2434),
 (3, 2435),
 (12, 2437),
 (14, 2438),
 (11, 2439),
 (13, 2441),
 (14, 2442),
 (2, 2444),
 (11, 2445),
 (13, 2447),
 (1, 2448),
 (15, 2449),
 (7, 2451),
 (11, 2452),
 (4, 2454),
 (5, 2455),
 (13, 2456),
 (11, 2458),
 (10, 2459),
 (3, 2461),
 (7, 2462),
 (4, 2464),
 (5, 2465),
 (6, 2467),
 (14, 2468),
 (15, 2469),
 (13, 2471),
 (6, 2472),
 (0, 2474),
 (5, 2475),
 (10, 2477),
 (3, 2478),
 (8, 2479),
 (10, 2481),
 (14, 2482),
 (5, 2484),
 (1, 2485),
 (11, 2486),
 (3, 2488),
 (11, 2489),
 (5, 2491),
 (0, 2492),
 (1, 2493),
 (7, 2495),
 (0, 2497),
 (4, 2498),
 (14, 2499),
 (12, 2501),
 (9, 2502),
 (4, 2504),
 (6, 2505),
 (1, 2506),
 (5, 2508),
 (4, 2509),
 (6, 2511),
 (10, 2512),
 (1, 2513),
 (13, 2515),
 (14, 2516

In [36]:
path_to_paradigm_files

'/nas/volume1/2photon/projects/gratings_phaseMod/20170927_CE059/FOV1_zoom3x/functional_subset/paradigm_files'

In [38]:
with open(os.path.join(path_to_paradigm_files, 'parsed_trials.pkl'), 'rb') as f:
    trialdict = pkl.load(f)

In [94]:
ref['frame_idxs'].index(2403) + 1

1443

In [50]:
ref['frame_idxs'][0:5]

[1, 2, 3, 6, 7]